<a href="https://colab.research.google.com/github/keripikkaneboo/Hands-On-Machine-Learning-O-Reilly-/blob/main/07.%20Chapter7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bab 7: Ensemble Learning dan Random Forests

Bab ini membahas **Ensemble Learning**, sebuah teknik di mana beberapa model (*predictors*) digabungkan untuk menghasilkan prediksi yang lebih baik daripada model tunggal mana pun. Idenya mirip dengan "kebijaksanaan orang banyak" (*wisdom of the crowd*): jawaban kolektif seringkali lebih baik daripada jawaban seorang ahli.

* **Voting Classifiers**: Cara paling sederhana untuk membuat *ensemble*. Beberapa model yang berbeda dilatih, dan prediksi akhir ditentukan oleh suara mayoritas.
    * ***Hard Voting***: Prediksi akhir adalah kelas yang paling banyak dipilih oleh para pengklasifikasi.
    * ***Soft Voting***: Menghitung rata-rata probabilitas kelas dari semua pengklasifikasi dan memilih kelas dengan probabilitas tertinggi. Biasanya memberikan performa lebih baik jika para pengklasifikasi dapat mengestimasi probabilitas.

* **Bagging dan Pasting**: Teknik *ensemble* di mana algoritma yang sama dilatih pada *subset* acak yang berbeda dari data training.
    * ***Bagging*** (*Bootstrap Aggregating*): Pengambilan sampel *subset* dilakukan **dengan** pengembalian (*with replacement*).
    * ***Pasting***: Pengambilan sampel *subset* dilakukan **tanpa** pengembalian (*without replacement*).
    * Teknik ini sangat baik untuk mengurangi varians model dan mudah diparalelkan.
    * ***Out-of-Bag (oob) Evaluation***: Karena *bagging* mengambil sampel dengan pengembalian, beberapa instance mungkin tidak pernah terpilih untuk dilatih oleh sebuah *predictor*. Instance-instance ini (disebut *out-of-bag*) dapat digunakan sebagai *validation set* tanpa perlu memisahkan data secara manual.

* **Random Forests**: Adalah sebuah *ensemble* dari Decision Trees, yang biasanya dilatih melalui metode *bagging*. Keunikan utamanya adalah selain mengambil sampel instance secara acak, ia juga mengambil sampel fitur secara acak di setiap *split node*. Hal ini menghasilkan pohon yang lebih beragam dan model yang lebih kuat.
    * ***Feature Importance***: Random Forest juga menyediakan cara mudah untuk mengukur pentingnya relatif dari setiap fitur.

* **Boosting**: Teknik *ensemble* di mana *predictor* dilatih secara sekuensial, dengan setiap *predictor* baru mencoba untuk memperbaiki kesalahan dari *predictor* sebelumnya.
    * ***AdaBoost (Adaptive Boosting)***: Fokus pada instance yang salah diklasifikasikan oleh *predictor* sebelumnya dengan cara meningkatkan bobotnya.
    * ***Gradient Boosting***: Melatih setiap *predictor* baru pada *residual error* (selisih antara target sebenarnya dan prediksi) dari *predictor* sebelumnya.

* **Stacking** (*Stacked Generalization*): Alih-alih menggunakan voting, sebuah model akhir (disebut *blender* atau *meta-learner*) dilatih untuk melakukan agregasi prediksi dari semua *predictor* di dalam *ensemble*.

### 1. Voting Classifier
Menggabungkan beberapa model klasifikasi yang berbeda untuk mendapatkan prediksi yang lebih baik.

```python
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Membuat dataset dan membaginya
X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Inisialisasi beberapa model
log_clf = LogisticRegression(random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(probability=True, random_state=42) # probability=True untuk soft voting

# Membuat Voting Classifier
# voting='hard' untuk mayoritas suara
# voting='soft' untuk rata-rata probabilitas
voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='soft'
)

voting_clf.fit(X_train, y_train)

# Membandingkan akurasi setiap model
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
```
Biasanya, akurasi `VotingClassifier` akan sedikit lebih tinggi daripada model individual terbaik.

### 2. Bagging dan Out-of-Bag (oob) Evaluation
Melatih banyak Decision Tree pada subset acak dari data training.

```python
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Membuat ensemble dengan 500 Decision Tree
# max_samples=100: setiap tree dilatih pada 100 instance acak
# bootstrap=True: sampling dengan pengembalian (bagging)
# oob_score=True: meminta evaluasi oob otomatis setelah training
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, oob_score=True, random_state=42)
bag_clf.fit(X_train, y_train)

# Skor oob adalah estimasi akurasi pada data yang tidak terlihat
print("OOB score:", bag_clf.oob_score_)

# Membandingkan dengan akurasi pada test set
y_pred = bag_clf.predict(X_test)
print("Accuracy on test set:", accuracy_score(y_test, y_pred))
```
Skor OOB seringkali menjadi estimasi yang baik untuk performa model pada *test set*.

### 3. Random Forest dan Feature Importance
`RandomForestClassifier` adalah versi yang lebih mudah digunakan dan dioptimalkan dari `BaggingClassifier` dengan `DecisionTreeClassifier`.

```python
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

# Inisialisasi Random Forest
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, random_state=42)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)
print("Random Forest accuracy:", accuracy_score(y_test, y_pred_rf))

# Menghitung feature importance pada dataset Iris
iris = load_iris()
rnd_clf_iris = RandomForestClassifier(n_estimators=500, random_state=42)
rnd_clf_iris.fit(iris["data"], iris["target"])

print("\nFeature importances on Iris dataset:")
for name, score in zip(iris["feature_names"], rnd_clf_iris.feature_importances_):
    print(name, score)
```
*Feature importance* menunjukkan fitur mana yang paling berpengaruh dalam membuat keputusan, yang sangat berguna untuk analisis.

### 4. Boosting
Contoh implementasi **AdaBoost**.

```python
from sklearn.ensemble import AdaBoostClassifier

# Menggunakan Decision Tree dangkal (stump) sebagai base estimator
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)
ada_clf.fit(X_train, y_train)

y_pred_ada = ada_clf.predict(X_test)
print("\nAdaBoost accuracy:", accuracy_score(y_test, y_pred_ada))
```
Boosting melatih model secara sekuensial, sehingga cenderung lebih lambat tetapi seringkali menghasilkan model yang sangat akurat.

